<a href="https://colab.research.google.com/github/Spazaldinho/DistributiveOverwatch/blob/main/CampQmindxOverwatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Welcome to Overwatch

**Please save a copy of this Colab so you can make edits.**

##0) Prepare for AI on Overwatch




In this section we are going to install all the dependencies we need and set the correct environment variables. Before you start this demo make sure you sign up for an Overwatch account. [Follow this link](https://app.overwatch.distributive.network/). You will also have to sign up for DCP account. The website will prompt you to do this.

###0.1 Install The Required Dependencies

We need to install overwatch-client so that we run inferences using overwatch. We also need onnx and onnxruntime. This is Microsoft's machine learning library which Overwatch uses.

In [ ]:
!python3 -m pip install overwatch-client --upgrade
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.1 MB/s eta 0:00:00


###0.2 Import The Necessary Packages

These are the packages we will need to use for the example below.

In [ ]:
from overwatch_client import HTTPClient
from matplotlib import pyplot as plt
from google.colab import files
import math
import glob
import os
import numpy as np
import cv2 as cv
import shutil
import onnx
import onnxruntime as ort
import sys

###0.3 Open The Client
This client is what our script will use to communicate with the Overwatch server.

In [ ]:
client = HTTPClient()

###0.4 Sign Into The Model Database
We need to authenticate ourselves in order to use the Overwatch service. If you have not signed up with Overwatch yet, please follow [this](https://app.overwatch.distributive.network/) link to sign up. Make sure you use your credentials that you signed up for Overwatch with.

In [ ]:
try:
    client.signin(email = 'mehedi@distributive.network',
                  password = '123456')
except Exception as e:
    print('Error: ' + str(e))

###0.5 Set Compute Group
When we request an inference to be performed, we need to assign it a compute group. A compute group is just a group of computers that work on the same group of tasks. Here we will generate the credentials to work on the ovwatch compute group. If we didn't supply credentials the inference would run on Distributive's Global compute group.

In [ ]:
joinKey     = "ovwatch"  # String
joinSecret  = "0UFRCfojif"  # String

###0.6 Download Our Model Zoo And Processing Code
This repository contains the models, processing code and plotting code for this demo. This command will download it for you and we will make use of it later.

In [ ]:
!git clone https://github.com/Distributive-Network/Overwatch-Model-Zoo.git

Cloning into 'Overwatch-Model-Zoo'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 104 (delta 39), reused 44 (delta 9), pack-reused 0
Receiving objects: 100% (104/104), 26.77 KiB | 6.69 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Error downloading object: FACES/FACES.onnx (34cd7e6): Smudge error: Error downloading FACES/FACES.onnx (34cd7e60aeff28744c657de7a3dc64e872d506741de66987f3426f2b79f88017): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/Overwatch-Model-Zoo/.git/lfs/logs/20231105T143031.180381598.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: FACES/FACES.onnx: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'



## 1) Beginner Example

###1.1 Model Selection
In this example, we will run an inference using Overwatch. To start off select a model that you would like to inference with. We have a list of offical models on our [GitHub](https://github.com/Distributive-Network/Overwatch-Model-Zoo). Pick a model you are interested in and copy its name from the directory. You can broswe through the models by checking out their READMEs.

Alternatively, you can do this by running the cell below. It will open up the Overwatch application in this notebook. You can log in using your credentials. Click on the models tab and you will see a list of models to choose from, along with their descriptions. Pick a model and copy its name.

In [ ]:
from IPython.display import IFrame

website_url = 'https://app.overwatch.distributive.network'

IFrame(website_url, width=1200, height=800)

###1.2 Set model to inference with
Now that you have a model name copied we can procede. Now we need to tell the Overwatch Client to inference with the model that you chose.

In [ ]:
modelName   = "MNIST"       # Replace your model name here

###1.3 Choose Files To Inference With
Choose which files you would like to inference with. They should be either pngs or jpgs. You can also set the image per slice, the number of images inferenced on per slice. If you set the image per slice to 1, that would mean 1 image would be inferenced per slice sent to each computer. If you don't have any images on your computer feel free to download some images from the web.

In [ ]:
from google.colab import files
uploaded    = files.upload()

imgPerSlice = 1            # Edit images per slice here
files   = {}

for upload_name, upload_data in uploaded.items():
  files[upload_name] = upload_data
numSlices = math.ceil(len(files)/imgPerSlice)
batchSize = math.floor(len(files)/numSlices)

print(f"Will infer on { len(files) } inputs, with { batchSize } inputs per slice, and a total of { numSlices } slices")

Saving mnist.png to mnist.png
Will infer on 1 inputs, with 1 inputs per slice, and a total of 1 slices


###1.4 Make The Inference Request
Here we will send the inference request through the Overwatch Client. No need to change anything.

In [ ]:
resp = client.infer(
    inputs             = files,
    model_name         = modelName,
    slice_batch        = batchSize,
    inference_id       = "detecting-faces...",
    compute_group_info = f"{joinKey}/{joinSecret}"
  )

print("Done Inferencing!")

KeyboardInterrupt: ignored

Here we will print the raw results of the inference. The results still need to be applied to their images so if the result don't make sense, don't worry!

In [ ]:
resp

###1.5 Plot The Results
Now that we have our raw results we can clean them up and plot them nicely so we can better understand the result of the inference we performed. Luckily we have some pre-written plotting code for you to use. In the cell below, we have written a script to import and implement the plotting script for an MNIST inference. If you are using a different model, you can change the script to use your model's plotting script. Anywhere in the cell below it says MNIST, change it to your model's name.

In [ ]:
sys.path.append('./Overwatch-Model-Zoo')
from MNIST.plot import MNIST_plot

Now run the plotting, the arguments will not change regardless of what model you are using.

In [ ]:
MNIST_plot(files, resp)

##2) Intermediate Example

###2.1 Pick A Model To Upload
In this example, we are going to learn how to upload your own model to the registry. To start off look through all the models in our model zoo and choose one you are interested in. The link to the github is right here:

[Overwatch Model Zoo](https://github.com/Distributive-Network/Overwatch-Model-Zoo)

Pick a model you are interested in and copy its name from the GitHub.

In [ ]:
model_selected = 'MNIST.onnx' # This should be the file name of the model. So if the model is called MNIST.onnx on our github it will also be named that here.
modelName      = 'MNIST_demo' # This is the name you want the model to have, so this can be anything. It will only be seen by that name on our model registry on our website.

###2.2 Upload The Model
This step will upload the model to Overwatch's model registry.

In [ ]:
path_prefix = 'Overwatch-Model-Zoo/' + model_selected.split('.')[0] + '/'

resp = client.register_model(
  model_name 		    = modelName,
  model_path 		    = path_prefix + model_selected,
  preprocess_path	  = path_prefix + 'preprocess.py',
  postprocess_path	= path_prefix + 'postprocess.py',
  language 		      = 'python',
  packages		      = ['numpy','opencv-python']
)


###2.3 Choose Files To Inference With
Choose which files you would like to inference with. They should be either pngs or jpgs. You can also set the image per slice, the number of images inferenced on per slice. If you set the image per slice to 1, that would mean 1 image would be inferenced per slice sent to each computer.

In [ ]:
from google.colab import files
uploaded    = files.upload()

imgPerSlice = 1            # Edit images per slice here
files   = {}

for upload_name, upload_data in uploaded.items():
  files[upload_name] = upload_data
numSlices = math.ceil(len(files)/imgPerSlice)
batchSize = math.floor(len(files)/numSlices)

print(f"Will infer on { len(files) } inputs, with { batchSize } inputs per slice, and a total of { numSlices } slices")

Saving person.jpg to person (3).jpg
Saving puppy.jpg to puppy (3).jpg
Saving puppy2.jpg to puppy2 (3).jpg
Will infer on 3 inputs, with 1 inputs per slice, and a total of 3 slices


###2.4 Make The Inference Request
Here we will send the inference request through the Overwatch Client.

In [ ]:
resp = client.infer(
    inputs        = files,
    model_name    = modelName,
    slice_batch   = batchSize,
    inference_id  = "detecting-faces...",
    compute_group_info = f"{joinKey}/{joinSecret}"
  )

print("Done Inferencing!")

Done Inferencing!


In [ ]:
print(resp)

{'Image.jpeg': {'output': [[0.0034078722819685936, 0.02747950702905655, 0.03438807651400566, 0.6144407987594604, 0.00162890728097409, 0.15403009951114655, 0.03564215824007988, 0.023539526388049126, 0.026166079565882683, 0.07927698642015457]]}}


###2.5 Plot The Results
Now that we have our raw results we can them up and plot them nicely so we can better understand the result of the inference we performed. Luckily we have some pre-written plotting code for you to use with plotting. In the cell below, we have written a script to import and implement the plotting script for an MNIST inference. If you are using a different model, you can change the script to use your model's plotting script. Anywhere in the cell below it says MNIST, change it to your model's name.

In [ ]:
sys.path.append('./Overwatch-Model-Zoo')
from MNIST.plot import MNIST_plot

Now run the plotting, the arguments will not change regardless of what model you are using.

In [ ]:
MNIST_plot(files, resp)

##3) Advanced Example

###3.1 Model Selection
In this example, we are going to learn how to write our own pre and post processing files. In case you have never done pre and post processing before, we will walk you through the steps for the MNIST model. If you are familiar with the process feel free to select any model and write your own processing scripts.

To start off look through all the models in our model zoo and choose one you are interested in. The link to the github is right here:

[Overwatch Model Zoo](https://github.com/Distributive-Network/Overwatch-Model-Zoo)

Pick a model you are interested in and copy its name from the directory.

###3.2 Download The Model
Make sure that you navigate through the colab file structure and download the .onnx file you are interested in. You can upload the model to Netron from your local machine and it will show you the structure of the model. This will help you to understand the structure of the model and how you can write a pre and post processing script.

###3.3 Look At Model Input And Output
Taking the model .onnx file you just downloaded, you can upload that to Netron in the cell below. This will let you visualize all the model layers. The layers we need to focus on are the input and output layers. They will be the first and second layers respectively. Looking at the tensor property of these layers we can see what shape they want. For instance if the property reads:

```
tensor: float32[1,1,28,28]
```

This means that the input will take a tensor that is of shape 1x1x28x28.

In [ ]:
from IPython.display import IFrame

website_url = 'https://netron.app'

IFrame(website_url, width=1200, height=800)

###3.4 Make Pre and Post Processing Files
Now that we understand the structure of the inputs and outputs of our model, we are going to make our pre and postprocessing files. The commands below make the files but it is up to us to fill them out.

In [ ]:
!touch preprocess.py
!touch postprocess.py

###3.5 Write preprocess.py
We will walk through writing our preprocess.py file together for an MNIST model. You can open up the preprocess.py file by opening up your file directory tab in Colab. It should be the last icon on the top left hand side of the screen.

Now that the file is open we can start writing. First things first we need to import the necessary libraries. In our case we will be using opencv and numpy.

```
import numpy as np
import cv2 as cv

```
We can now create our preprocessing function, it must be called preprocessing as shown below. It also must have the arguments `bytes` and `inputNames`. `bytes` contains the data that will be inferenced upon. It is stored in a buffer. In the case of MNIST, the data is images of handwritten digits. `inputNames` has the list of names that the ONNX model is expecting for the data it recieves. You can think of it as the key to open the ONNX model. In this case, there is only one input to the model so we only pay attention to the first element in that list.

```
def preprocess(bytes, inputNames):
```

Now we need to create the object that will be fed to the ONNX model for inferencing. ONNX models take a dictionary so we can create the `feeds` object. We also will need the name of the input to send to the ONNX model. This can be found from the `inputNames` variable as shown below.


```
  feeds = dict()
  inputNames = str(inputNames[0])
```

Because the data is stored in a buffer in the `bytes` variable we can load the data from the buffer with `np.frombuffer`. We can then decode that into a colour image using opencv's `cv.imdecode` function.

```
  bytesInput = np.frombuffer( bytes, dtype=np.uint8)
  image = cv.imdecode(bytesInput, cv.IMREAD_COLOR)
```

The MNIST model takes in an image of size **1x1x28x28**.  The reason for this is that each of the images have a size of **28x28**. This means that each side of an MNIST picture has 28 pixels. The MNIST images are also grayscale, this means that the typical 3 colour channels of red, green and blue get condensed to a single grayscale channel. The model also only expects one image at a time so thus the model input tensor dimensions are **1x1x28x28**. Code to convert an arbitrarily sized colour image to a grayscale image is below. The pixel values are also divided by 255 because typical pixel values are between 0-255 but the model expects values between 0 and 1.

```
  image   = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
  image = image / 255
  image = cv.resize(image, (28,28))
  image = np.reshape(np.asarray(image.astype(np.float32)), (1, 1, 28, 28))
```

Now we just need to return the `feeds` dict. The dict key should be the inputNames and the value should be the image.

```
  feeds[inputNames] = image
  return feeds
```

Save this file and it will be good to go.

###3.6 Write postprocess.py
We will walk through writing our postprocess.py file together for an MNIST model. First things first we need to import the necessary libraries. In our case we will be using numpy.

```
import numpy as np
```
We can now create our preprocessing function, it must be called postprocessing as shown below. It also must have the arguments `out`, `labels` and `outputNames`. `out` contains the results of the inference and is the what we will be analyzing.
```
def postprocess(out, labels, outputNames):
```

Now we need to create the object that will be fed to the ONNX model for inferencing. ONNX models take a dictionary so we can create the `outputs` object.


```
   outputs = dict()
```

The MNIST Post processing is relatively simple. In the `out` dict, the output of the model is stored. The output is **1x10** tensor with each of the 10 outputs representing a number betwen 0 and 9. In order to find the probability of each number being in the image being inferenced, the softmax function is applied to the results.
```
  probabilities = np.exp(out[outputNames[0]]) / np.sum(np.exp(out[outputNames[0]]), axis=1, keepdims=True)
```
Those results are then returned to the user.

```
  outputs['output'] = probabilities.tolist()
  return outputs
```

Save this file and it will be ready for upload to Overwatch.

###3.7 Upload The Model
Like in the previous example we can upload our model to Overwatch's model registry.

In [ ]:
model_selected = 'MNIST.onnx' # This should be the file name of the model. So if the model is called MNIST.onnx on our github it will also be named that here.
modelName      = 'MNIST_demo' # This is the name you want the model to have, so this can be anything. It will only be seen by that name on our model registry on our website.

In [ ]:
path_prefix = 'Overwatch-Model-Zoo/' + model_selected.split('.')[0] + '/'

resp = client.register_model(
  model_name 		    = modelName,
  model_path 		    = path_prefix + model_selected,
  preprocess_path	  = 'preprocess.py',
  postprocess_path	= 'postprocess.py',
  language 		      = 'python',
  packages		      = ['numpy','opencv-python']
)


###3.8 Inference

Upload files and inference on them with your new model through overwatch.


In [ ]:
from google.colab import files
uploaded    = files.upload()

imgPerSlice = 1            # Edit images per slice here
files   = {}

for upload_name, upload_data in uploaded.items():
  files[upload_name] = upload_data
numSlices = math.ceil(len(files)/imgPerSlice)
batchSize = math.floor(len(files)/numSlices)

print(f"Will infer on { len(files) } inputs, with { batchSize } inputs per slice, and a total of { numSlices } slices")

Perform the inference

In [ ]:
resp = client.infer(
    inputs        = files,
    model_name    = modelName,
    slice_batch   = batchSize,
    inference_id  = "detecting-faces...",
    compute_group_info = f"{joinKey}/{joinSecret}"
  )

print("Done Inferencing!")

###3.9 Plot The Results
Now that we have our raw results, we can clean them up and plot them nicely so we can better understand the result of the inference we performed. Luckily we have some pre-written plotting code for you to use with plotting. In the cell below, we have written a script to import and implement the plotting script for an MNIST inference. If you are using a different model, you can change the script to use your model's plotting script. Anywhere in the cell below it says MNIST, change it to your model's name.

In [ ]:
sys.path.append('./Overwatch-Model-Zoo')
from MNIST.plot import MNIST_plot

In [ ]:
MNIST_plot(files, resp)

##4) Expert Example

###4.1 Model Conversion
In this example we will go through the basics of learning to convert models to the ONNX format. While ONNX now lets you train models, the PyTorch and Tensorflow frameworks are the most popular. Here are some guides on how to convert from those frameworks into ONNX.
1. [PyTorch to ONNX Conversion](https://medium.com/deci-ai/tutorial-converting-a-pytorch-model-to-onnx-format-f1bbce156d2a)
2. [Tensorflow to ONNX](https://github.com/onnx/tensorflow-onnx)

We left some space below for you to convert your model.

In [ ]:
# convert your model to ONNX here

###4.2 Pre And Post Processing
Now that you have converted your model, remember you still have to make the pre and post processing files as shown in the Advanced Example. You can always use the [Netron tool](netron.app) to help you! Make sure these scripts are customized for your new model and augment the data in the way you intend, otherwise you will get the wrong results.

In [ ]:
!touch preprocess.py
!touch postprocess.py

###4.3 Upload And Inference With Your Model
Look back the Intermediate and Beginner Examples to remember how to upload and inference with your model using the client. You can also write your own plotting script to the better visualize your results after they are returned by Overwatch.

In [ ]:
# upload your model here

In [ ]:
# inference with your model here

In [ ]:
# plot your results here

##5) Wizard Example

If you have gotten here that is amazing! In this example you will train your own custom model in whatever framework you are most comfortable in! Then you will convert the model to ONNX (unless you trained it in ONNX), write the pre and processing scripts, upload the model with the client and then inference with the client. This example will bring together everything you have learned in previous examples. Here are the steps broken down:


1.   Train your own model in whichever framework you want i.e. Tensorflow, PyTorch, ONNX
2.   Convert your model to be in ONNX. You will have learned how to do this in the Expert Example.
3.   Write the pre and post processing scripts for your model as shown in the Advanced Example.
4.   Upload your model and the accompanying processing scripts as shown in the Intermediate Example.
5.   Perform an inference with the newly uploaded model as shown in the Beginner Example.

Good luck with this challenge!

